Importing the appropriate packages, will import more later.

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import json
import tensorflow as tf
import tensorflow.keras as keras
import h5py

Importing the Green+2020 model below.

In [3]:
nn_model = tf.keras.models.load_model('/arc/home/aydanmckay/green2020-stellar-model/green2020_nn_model.h5')
nn_model.summary()

Model: "green2020_stellar_photometry_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 theta (InputLayer)             [(None, 3)]          0           []                               
                                                                                                  
 stellar_model_hidden_1 (Dense)  (None, 32)          128         ['theta[0][0]']                  
                                                                                                  
 E (InputLayer)                 [(None, 1)]          0           []                               
                                                                                                  
 R (Dense)                      (None, 13)           52          ['theta[0][0]']                  
                                                                 

### Normalization of spectroscopic labels

Before passing spectroscopic labels, $\theta$, into the neural network, we have to normalize them. We scale and shift $\left( T_{\mathrm{eff}}, \log g, \left[ \mathrm{Fe} / \mathrm{H} \right] \right)$ so that they are typically small numbers. Specifically, we scale and shift them based on the median and standard deviation of the training data, which we load from one of the provided data files.

Definitely something to look into, want to see if these scales and shifts are fine or if we should (probably) be using our own training data instead. This will be the case when we expand on the number of parameters.

In [5]:
with open('/arc/home/aydanmckay/green2020-stellar-model/green2020_theta_normalization.json', 'r') as f:
    d = json.load(f)

theta_med = np.array(d['theta_med'])
theta_std = np.array(d['theta_std'])

def normalize_theta(theta):
    """
    Normalizes spectroscopic labels, so that they can be input to neural
    network.

    Inputs:
      theta (numpy.ndarray): Array containing (T_eff, logg, [Fe/H]) in
          units of (K, dex, dex) for each star. Shape = (# of stars, 3).
    
    Returns:
      Normalized theta. The normalization is based on the mean and standard
      deviation of theta in the training dataset.
    """
    # theta_med = np.array([5372.02, 4.06696, -0.246082])
    # theta_std = np.array([902.802, 0.889269, 0.373921])
    x = (theta - theta_med[None,:]) / theta_std[None,:]
    return x

In [8]:
with h5py.File('/arc/home/aydanmckay/green2020-stellar-model/green2020_test_data_small.h5', 'r') as f:
    d = f['data'][:]       # All the data needed to train or test the model
    r_fit = f['r_fit'][:]  # The reddening inferred using the trained model
    r_var = f['r_var'][:]  # The variance of the inferred reddening

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'green2020_test_data_small.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)